# Lesson 5: CD for ML (Deploy to Prod)

**Module 6: Monitoring & CI/CD**  
**Estimated Time**: 60 mins  
**Difficulty**: Advanced

---

## 🎯 Learning Objectives

By the end of this lesson, you will:

✅ Understand **Continuous Deployment** (CD) strategies.  
✅ Automate **Docker Image Building** and Pushing to Docker Hub.  
✅ Learn about **Rolling Updates** and **Blue/Green Deployment**.  
✅ Understand the **GitOps** philosophy (ArgoCD).  

---

## 📚 Table of Contents

1. [The "Deploy" Button](#1-deploy)
2. [Deployment Strategies](#2-strategies)
3. [Hands-On: Automating Docker Push](#3-hands-on)
4. [GitOps: The Modern Way](#4-gitops)
5. [Interview Preparation](#5-interview)

---

### 🛠️ Setup
This lesson involves writing YAML files.

## 1. The "Deploy" Button

Once CI passes (tests are green), we want to ship it.

### The CD Pipeline Steps:
1. **Tag**: Version the potential release (e.g., `v1.0.1`).
2. **Build**: Create a Docker Image (`myapp:v1.0.1`).
3. **Push**: Upload to DockerHub/ECR.
4. **Deploy**: Tell Kubernetes to update the Deployment to use image `v1.0.1`.

## 2. Deployment Strategies

How do we update the running application without downtime?

| Strategy | Description | Pros | Cons |
|---|---|---|---|
| **Recreate** | Kill old V1, then start new V2. | Simple state. | Downtime during switch. |
| **Rolling Update** | Replace 1 pod at a time (V1 -> V1,V2 -> V2). | No downtime. | Mix of V1/V2 answering requests. |
| **Blue/Green** | Spin up full V2 cluster alongside V1. Switch Load Balancer. | Instant rollback. | 2x cost (Double resources). |
| **Canary** | Send 10% traffic to V2. If healthy, increase to 100%. | Safe testing. | Complex setup. |

## 3. Hands-On: Automating Docker Push

We will write a `cd.yaml` workflow that triggers only when we push a **Tag** (e.g., `v*`).

In [ ]:
%%writefile cd_pipeline.yaml
name: ML Pipeline CD

# Trigger on Tags only
on:
  push:
    tags:
      - 'v*'

jobs:
  build-and-push:
    runs-on: ubuntu-latest
    steps:
    - uses: actions/checkout@v3

    - name: Log in to Docker Hub
      uses: docker/login-action@v2
      with:
        username: ${{ secrets.DOCKER_USERNAME }}
        password: ${{ secrets.DOCKER_PASSWORD }}

    - name: Build and Push Docker image
      uses: docker/build-push-action@v4
      with:
        context: .
        push: true
        tags: myuser/myapp:latest,myuser/myapp:${{ github.ref_name }}

## 4. GitOps: The Modern Way

Instead of the CD pipeline running `kubectl apply`, we use **GitOps**.

1. **CD Pipeline**: Builds Docker image and updates a `values.yaml` file in a separate git repo (Config Repo).
2. **ArgoCD** (running inside K8s): Watches the Config Repo.
3. **Sync**: ArgoCD sees the change and pulls the new image into the cluster.

*Benefit*: Your cluster state is always exactly what is in Git. No "drift".

## 5. Interview Preparation

**Q1: What is Blue/Green deployment?**  
*A1: It's a strategy where you run two identical environments. Blue is live. You deploy to Green. Once tested, you switch the router to point to Green. This allows instant rollback manually or automatically.*

**Q2: Why use GitOps instead of just `kubectl apply` in CI?**  
*A2: Security and Visibility. With GitOps, CI doesn't need admin access to the cluster (Credentials stay in cluster). Also, the Git history becomes the audit log of every change made to the cluster.*